In [1]:
# Import the requests library.
import requests
import numpy as np
import pandas as pd
# Import the requests library.
import requests
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
# Dependencies for the wine API
import urllib
import json
# Import the API key.
from config import weather_api_key

Token = 'tylRGHbxqiZUDNymMowCBFCuEqeULiWk'
API_Token='Token d8f94b93417333267ca0edcca7dc14a9e035e1bb'

In [2]:
# Import the API key.
headers = {
    'accept': 'application/json',
    'authorization': API_Token
    }

# Red Wine in USA

In [3]:
req_red_USA = urllib.request.Request('https://api.globalwinescore.com/globalwinescores/latest/?color=red&limit=5000&country=Usa', data=None, headers=headers, origin_req_host=None, unverifiable=False, method=None)
res_red_USA = urllib.request.urlopen(req_red_USA).read().decode()
obj_red_USA = json.loads(res_red_USA)
#read json into DF
Red_USA_df = pd.DataFrame.from_dict(obj_red_USA['results'], orient='columns')
# To delete the column without having to reassign df you can do:
Red_USA_df.drop([ 'wine_slug', 'appellation_slug',  'date',  'lwin', 'lwin_11'],axis=1, inplace=True)
#'wine_id','wine_type','classification',
#Drop NA values
# Red_USA_df = Red_USA_df.dropna()
#convert vintage to date type
Red_USA_df['vintage']= pd.to_datetime(Red_USA_df['vintage']).dt.year
Red_USA_df['regions']=Red_USA_df['regions'].str.get(0)

# White Wine in USA

In [4]:
req_white_USA = urllib.request.Request('https://api.globalwinescore.com/globalwinescores/latest/?color=white&limit=5000&country=Usa', data=None, headers=headers, origin_req_host=None, unverifiable=False, method=None)
res_white_USA = urllib.request.urlopen(req_white_USA).read().decode()
obj_white_USA = json.loads(res_white_USA)
#read json into DF
white_USA_df = pd.DataFrame.from_dict(obj_white_USA['results'], orient='columns')
# To delete the column without having to reassign df you can do:
white_USA_df.drop([ 'wine_slug', 'appellation_slug',  'date',  'lwin', 'lwin_11'],axis=1, inplace=True)
#'wine_id','wine_type','classification',
#Drop NA values
# white_USA_df = white_USA_df.dropna()
#convert vintage to date type
white_USA_df['vintage']= pd.to_datetime(white_USA_df['vintage']).dt.year
white_USA_df['regions']=white_USA_df['regions'].str.get(0)

# Weather

In [5]:
#California_GHCND:
Nappa_Valley_g='GHCND:USC00046074'
# Washington_GHCND:
Walla_Walla_Valley_g='GHCND:GHCND:USC00457015'
# Oregon_GHCND:
Willamette_Valley_g='GHCND:USW00024232'

#California_LAT:
Nappa_Valley_lat='38.2777'
# Washington_Lat:
Walla_Walla_Valley_lat='46.3119'
# Oregon_lat:
Willamette_Valley_lat='44.905'

#California_Long:
Nappa_Valley_long='-122.2647'
# Washington_Long:
Walla_Walla_Valley_long= '-119.2633'
# Oregon_Long:
Willamette_Valley_long='-123.0011'

In [6]:
station_id = (Nappa_Valley_g, Walla_Walla_Valley_g, Willamette_Valley_g)
station_lat=(Nappa_Valley_lat, Walla_Walla_Valley_lat, Willamette_Valley_lat)
station_long=(Nappa_Valley_long, Walla_Walla_Valley_long, Willamette_Valley_long)
#find the weather station
# https://www.ncdc.noaa.gov/cdo-web/datatools/findstation

In [7]:
#Create average temperature
dates_temp = []
temps = []
station_id=['GHCND:USC00046074','GHCND:GHCND:USC00457015','GHCND:USW00024232']


#for each year from 2015-2019 ...
for year in range(2015, 2016):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    #get the date field from all average temperature readings
    dates_temp += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps += [item['value'] for item in avg_temps]

#initialize dataframe
df_temp = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
#convert to degrees F
df_temp['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps]
df_temp = df_temp.set_index('date').resample('M').mean()

working on year 2015


In [8]:
#initialize lists to store data
dates_prcp = []
prcp = []

#for each year from 2015-2019 ...
for year in range(2015, 2016):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are Precipitation temperature readings
    avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all average Precipitation readings
    dates_prcp += [item['date'] for item in avg_prcp]
    #get the actual average Precipitation from all average Precipitation readings
    prcp += [item['value'] for item in avg_prcp]

#initialize dataframe
df_prcp = pd.DataFrame()

#populate date and average Precipitation fields (cast string date to datetime
df_prcp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp]
#convert to mm to inches
df_prcp['avgPrcp'] = [float(v)*0.0393701 for v in prcp]
df_prcp = df_prcp.set_index('date').resample('M').mean()

working on year 2015


In [9]:
df_temp['avgPrcp']=df_prcp['avgPrcp']
# df_temp

,avgTemp,avgPrcp
date,,
2015-01-31,59.418065,0.280671
2015-02-28,62.092143,0.085771
2015-03-31,66.083871,0.158750
2015-04-30,64.952000,0.073491
2015-05-31,63.604516,0.247651
2015-06-30,69.668000,0.003937
2015-07-31,73.295484,0.173990
2015-08-31,75.635484,0.000000
2015-09-30,77.396000,0.316273


In [10]:
Nappa=['Nappa Valley',
'Rutherford',
'Oakville',
'Howell Mountain',
'Sonoma County',
'Stags Leap District',
'Russian River Valley',
'St Helena',
'Sonoma Coast',
'Carneros',
'Santa Cruz Mountains',
'Knights Valley',
'California',
'Alexander Valley',
'Spring Mountain District',
'Dry Creek Valley',
'Mount Veeder',
'Yountville',
'Diamond Mountain District',
'Santa Lucia Highlands',
'Green Valley',
'Paso Robles',
'Oak Knoll District',
'Coombsville',
'Santa Ynez Valley',
'Atlas Peak',
'Chalk Hill',
'Amador County',
'Calistoga',
'Napa County',
'Sonoma Valley',
'Los Carneros',
'Anderson Valley',
'Bennett Valley',
'Mendocino Ridge',
'St Rita Hills',
'Monterey',
'Monticello',
'Mount Harlan',
'North Coast',
'San Luis Obispo County',
'Santa Mario Valley',
'Sierra Foothills']

Washington=['Walla Walla Valley',
'Columbia Valley',
'Red Mountain',
'Yakima Valley',
'Horse Heaven Hills',
'Washington',
'Snipes Mountains']

Oregon=['Willamette Valley',
'Dundee Hills',
'Eola-Amity Hills',
'Oregon',
'Ribbon Ridge',
'Chehalem Mountains']

In [11]:
white_USA_df.head(5)

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count
0,"Ridge Vineyards, Monte Bello Chardonnay, White...",120785,Santa Cruz Mountains,White,dry,California,Usa,None,2016,False,94.39,B,3
1,"Stags' Leap Winery, Chardonnay, White, Napa Va...",129823,Napa Valley,White,dry,California,Usa,None,2016,False,89.83,A+,3
2,"Joseph Phelps Vineyards, Freestone Chardonnay,...",89569,Sonoma Coast,White,dry,California,Usa,None,2016,False,90.73,C+,4
3,"Crossbarn By Paul Hobbs, Chardonnay, White, So...",47377,Sonoma Coast,White,dry,California,Usa,None,2016,False,90.39,B+,3
4,"Mount Eden Vineyards, Chardonnay, White, Santa...",107658,Santa Cruz Mountains,White,dry,California,Usa,None,2015,False,92.22,B,4


In [12]:
# for i in white_USA_df:
#     if white_USA_df['regions']== 'Calfornia' and white_USA_df['vintage']== 2016:
#         white_USA_df['March']==
    

In [13]:
# df_temp

,avgTemp,avgPrcp
date,,
2015-01-31,59.418065,0.280671
2015-02-28,62.092143,0.085771
2015-03-31,66.083871,0.158750
2015-04-30,64.952000,0.073491
2015-05-31,63.604516,0.247651
2015-06-30,69.668000,0.003937
2015-07-31,73.295484,0.173990
2015-08-31,75.635484,0.000000
2015-09-30,77.396000,0.316273


In [14]:
sorted_white_USA_df=white_USA_df.sort_values(by=['vintage'], ascending=False)
sorted_white_USA_df.head(2)

,wine,wine_id,appellation,color,wine_type,regions,country,classification,vintage,is_primeurs,score,confidence_index,journalist_count
0,"Ridge Vineyards, Monte Bello Chardonnay, White...",120785,Santa Cruz Mountains,White,dry,California,Usa,None,2016,False,94.39,B,3
13,"Aubert Wines, Larry Hyde & Sons Vineyard Chard...",6880,Carneros,White,dry,California,Usa,None,2016,False,96.98,B+,3


In [15]:
df=white_USA_df.groupby('regions')['vintage'].apply(list).to_dict()
df

{'California': [2016,
  2016,
  2016,
  2016,
  2015,
  2016,
  2016,
  2015,
  1993,
  1998,
  2016,
  2016,
  2015,
  2016,
  2015,
  2016,
  2016,
  2016,
  2015,
  2016,
  2016,
  2007,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2014,
  2016,
  2011,
  2015,
  2015,
  2015,
  2015,
  2015,
  2016,
  2016,
  2016,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2014,
  2015,
  2016,
  2014,
  2013,
  2015,
  2015,
  2015,
  2015,
  2015,
  2015,
  2016,
  2014,
  2014,
  2015,
  2015,
  2015,
  2015,
  2015,
  2013,
  2013,
  2015,
  2015,
  2013,
  2014,
  2014,
  2014,
  2015,
  2009,
  2014,
  2013,
  2014,
  2014,
  2014,
  2015,
  2014,
  2013,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2015,
  2015,
  2013,
  2015,
  2015,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2014,
  2015,
  2015,
  2015,
  2010,
  2012,
  2014,
  2013,
  2014,
  2014,
  2010,
  2013,
  

# California Avg Temp and Precipitation

In [16]:
#Create average temperature
dates_temp = []
temps = []

#for each year from 2015-2019 ...
for year in range(2013, 2017):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    #get the date field from all average temperature readings
    dates_temp += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps += [item['value'] for item in avg_temps]

#initialize dataframe
df_temp = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
#convert to degrees F
df_temp['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps]
df_temp = df_temp.set_index('date').resample('M').mean()

working on year 2013
working on year 2014
working on year 2015
working on year 2016


In [17]:
### initialize lists to store data
dates_prcp = []
prcp = []

#for each year from 2015-2019 ...
for year in range(1992, 2017):
    year = str(year)
#     print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are Precipitation temperature readings
    avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all average Precipitation readings
    dates_prcp += [item['date'] for item in avg_prcp]
    #get the actual average Precipitation from all average Precipitation readings
    prcp += [item['value'] for item in avg_prcp]

#initialize dataframe
df_prcp = pd.DataFrame()

#populate date and average Precipitation fields (cast string date to datetime
df_prcp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp]
#convert to mm to inches
df_prcp['avgPrcp'] = [float(v)*0.0393701 for v in prcp]
df_prcp = df_prcp.set_index('date').resample('M').mean()

In [18]:
df_temp['avgPrcp']=df_prcp['avgPrcp']

# Washington Avg Temp and Precipitation

In [19]:
#Create average temperature
dates_temp_W = []
temps_W = []

#for each year from 2015-2019 ...
for year in range(2013, 2017):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    #get the date field from all average temperature readings
    dates_temp_W += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps_W += [item['value'] for item in avg_temps]

#initialize dataframe
df_temp_W = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_W]
#convert to degrees F
df_temp_W['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_W]
df_temp_W = df_temp_W.set_index('date').resample('M').mean()
# df_temp_W

working on year 2013
working on year 2014
working on year 2015
working on year 2016


,avgTemp
date,
2013-04-30,62.678000
2013-05-31,67.123226
2013-06-30,68.300000
2013-07-31,70.316774
2013-08-31,70.432903
2013-09-30,72.326000
2013-10-31,65.950323
2013-11-30,62.612000
2013-12-31,57.072258


In [20]:
### initialize lists to store data
dates_prcp_W = []
prcp_W = []

#for each year from 2015-2019 ...
for year in range(1992, 2017):
    year = str(year)
#     print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are Precipitation temperature readings
    avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all average Precipitation readings
    dates_prcp_W += [item['date'] for item in avg_prcp]
    #get the actual average Precipitation from all average Precipitation readings
    prcp_W += [item['value'] for item in avg_prcp]

#initialize dataframe
df_prcp_W = pd.DataFrame()

#populate date and average Precipitation fields (cast string date to datetime
df_prcp_W['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_W]
#convert to mm to inches
df_prcp_W['avgPrcp'] = [float(v)*0.0393701 for v in prcp_W]
df_prcp_W = df_prcp_W.set_index('date').resample('M').mean()

In [21]:
df_temp_W['avgPrcp']=df_prcp_W['avgPrcp']
# df_temp_W

,avgTemp,avgPrcp
date,,
2013-04-30,62.678000,0.411350
2013-05-31,67.123226,0.109220
2013-06-30,68.300000,0.227034
2013-07-31,70.316774,0.000000
2013-08-31,70.432903,0.000000
2013-09-30,72.326000,0.223097
2013-10-31,65.950323,0.000000
2013-11-30,62.612000,0.389628
2013-12-31,57.072258,0.244366


#  Oregon Avg Temp and Precipitation

In [22]:
#Create average temperature
dates_temp_O = []
temps_O = []

#for each year from 2015-2019 ...
for year in range(2013, 2017):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']
    #get the date field from all average temperature readings
    dates_temp_O += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps_O += [item['value'] for item in avg_temps]

#initialize dataframe
df_temp_O = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime
df_temp_O['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp_O]
#convert to degrees F
df_temp_O['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps_O]
df_temp_O = df_temp_O.set_index('date').resample('M').mean()

working on year 2013
working on year 2014
working on year 2015
working on year 2016


In [23]:
### initialize lists to store data
dates_prcp_O = []
prcp_O = []

#for each year from 2015-2019 ...
for year in range(1992, 2017):
    year = str(year)
#     print('working on year '+year)
    
    #make the api call
    r = requests.get(f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&stationid=GHCND:USC00046074&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are Precipitation temperature readings
    avg_prcp = [item for item in d['results'] if item['datatype']=='PRCP']
    #get the date field from all average Precipitation readings
    dates_prcp_O += [item['date'] for item in avg_prcp]
    #get the actual average Precipitation from all average Precipitation readings
    prcp_O += [item['value'] for item in avg_prcp]

#initialize dataframe
df_prcp_O = pd.DataFrame()

#populate date and average Precipitation fields (cast string date to datetime
df_prcp_O['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_prcp_O]
#convert to mm to inches
df_prcp_O['avgPrcp'] = [float(v)*0.0393701 for v in prcp_O]
df_prcp_O = df_prcp_O.set_index('date').resample('M').mean()

In [24]:
df_temp_O['avgPrcp']=df_prcp_O['avgPrcp']
# df_temp_O

,avgTemp,avgPrcp
date,,
2013-04-30,62.678000,0.411350
2013-05-31,67.123226,0.109220
2013-06-30,68.300000,0.227034
2013-07-31,70.316774,0.000000
2013-08-31,70.432903,0.000000
2013-09-30,72.326000,0.223097
2013-10-31,65.950323,0.000000
2013-11-30,62.612000,0.389628
2013-12-31,57.072258,0.244366
